In [ ]:
# Main Game Loop with Levels
import os
import sys
import pygame
import Configure_The_Simulation as config
from Simulated_Functions import (
    get_outer_walls, get_default_inner_walls, draw_player, show_game_over, load_player_image, computer_rect, asset_path, Enemy, count_objectives, show_escape_message
)

# Each level has its computer, enemies, and custom walls
LEVELS = [
    {
        "computer_pos": (30, 30),
        "player_start": (530, 530),
        "enemies": [
            {"x": 150, "y": 100, "movement": "vertical", "speed": 200},
            {"x": 300, "y": 340, "movement": "horizontal", "speed": 400},
            {"x": 300, "y": 200, "movement": "vertical", "speed": 200},
            {"x": 450, "y": 240, "movement": "horizontal", "speed": 350},
        ],
        "walls": None 
    },
    {
        "computer_pos": (530, 30),
        "player_start": (280, 50),
        "enemies": [
            {"x": 220, "y": 150, "movement": "horizontal", "speed": 50}, #
            {"x": 150, "y": 220, "movement": "horizontal", "speed": 150}, #
            {"x": 100, "y": 430, "movement": "horizontal", "speed": 40}, #
            {"x": 100, "y": 300, "movement": "horizontal", "speed": 40}, # 
            {"x": 30, "y": 30, "movement": "vertical", "speed": 30}, #
            {"x": 400, "y": 500, "movement": "vertical", "speed": 150}, #
            {"x": 500, "y": 100, "movement": "vertical", "speed": 150}, #
            {"x": 510, "y": 300, "movement": "horizontal", "speed": 120}, #
            {"x": 510, "y": 450, "movement": "horizontal", "speed": 180}, #
        ],
    
        "walls": [
            pygame.Rect(180, 0, 20, 200),
            pygame.Rect(350, 0, 20, 300),
            pygame.Rect(90, 280, 270, 20),
            pygame.Rect(90, 0, 20, 200),
            pygame.Rect(160, 280, 20, 200),
            pygame.Rect(160, 300, 210, 200),
        ]
    },
    {
        "computer_pos": (275, 250),
        "player_start": (300, 550),
        "enemies": [
            {"x": 205, "y": 150, "movement": "vertical", "speed": 800},
            {"x": 520, "y": 100, "movement": "vertical", "speed": 800},
            {"x": 340, "y": 100, "movement": "vertical", "speed": 100},
            {"x": 340, "y": 250, "movement": "vertical", "speed": 200},
            {"x": 90, "y": 20, "movement": "horizontal", "speed": 350},
            {"x": 30, "y": 500, "movement": "vertical", "speed": 200},
            {"x": 30, "y": 175, "movement": "horizontal", "speed": 150},
        ],
    
        "walls": [
            pygame.Rect(90, 480, 410, 20),
            pygame.Rect(90, 500, 20, 100),
            pygame.Rect(90, 400, 100, 20),
            pygame.Rect(270, 400, 230, 20),
            pygame.Rect(0, 320, 190, 20),
            pygame.Rect(400, 320, 100, 20),
            pygame.Rect(270, 320, 50, 20),
            pygame.Rect(270, 200, 225, 20),
            pygame.Rect(315, 200, 20, 140),
            pygame.Rect(270, 80, 20, 140)

        ]
    },
    {
        "computer_pos": (275, 50),
        "player_start": (300, 550),
        "enemies": [
           {"x": 150, "y": 150, "movement": "vertical", "speed": 500},
           {"x": 400, "y": 150, "movement": "vertical", "speed": 300},
            {"x": 275, "y": 300, "movement": "horizontal", "speed": 400},
            {"x": 100, "y": 400, "movement": "horizontal", "speed": 200},
            {"x": 450, "y": 400, "movement": "horizontal", "speed": 200},
            {"x": 30, "y": 250, "movement": "vertical", "speed": 150},
            {"x": 520, "y": 250, "movement": "vertical", "speed": 150},
            {"x": 275, "y": 100, "movement": "horizontal", "speed": 400},
            {"x": 100, "y": 500, "movement": "vertical", "speed": 300},
            {"x": 450, "y": 500, "movement": "vertical", "speed": 300},
            {"x": 275, "y": 400, "movement": "horizontal", "speed": 600},
            {"x": 30, "y": 100, "movement": "vertical", "speed": 400},
            {"x": 520, "y": 100, "movement": "vertical", "speed": 200},
            {"x": 150, "y": 300, "movement": "vertical", "speed": 250},
            {"x": 400, "y": 300, "movement": "vertical", "speed": 250},
            {"x": 100, "y": 200, "movement": "horizontal", "speed": 150},
            {"x": 450, "y": 200, "movement": "horizontal", "speed": 150},
            {"x": 275, "y": 250, "movement": "horizontal", "speed": 350},
            {"x": 100, "y": 50, "movement": "horizontal", "speed": 400},
            {"x": 300, "y": 50, "movement": "vertical", "speed": 400},
            {"x": 450, "y": 50, "movement": "horizontal", "speed": 400},

        ],
    
        "walls": [

        ]
    }
]

def load_level(screen, level_index):
    level_data = LEVELS[level_index]

    # Computer location
    comp_rect = pygame.Rect(*level_data["computer_pos"], 40, 40)

    # Enemy objects
    enemies = [
        Enemy(e["x"], e["y"], movement=e["movement"], speed=e["speed"])
        for e in level_data["enemies"]
    ]

    player_start = level_data.get("player_start", None)

    walls = get_outer_walls()

    if level_data.get("walls") is not None:
        walls += level_data["walls"]
    else:
        walls += get_default_inner_walls()

    return comp_rect, enemies, walls, player_start

def load_level_safe(level_index, screen):
    comp_rect, enemies, walls, player_start = load_level(screen, level_index)

    # Reset all enemies to starting positions
    for e in enemies:
        e.reset_position()
        e.direction = 1
        
    # Player starting position
    player_x, player_y = player_start
    computer_cooldown = False

    return comp_rect, enemies, walls, player_x, player_y, computer_cooldown, player_start

def wait_for_escape():
    esc_pressed = False
    while not esc_pressed:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if event.type == pygame.KEYDOWN and event.key == pygame.K_ESCAPE:
                esc_pressed = True
        pygame.time.wait(10)

def main():
    pygame.init()
    screen = pygame.display.set_mode((config.WIDTH, config.HEIGHT))
    pygame.display.set_caption("3x3 Adventure Grid")
    clock = pygame.time.Clock()

    # Load assets
    music_file = asset_path("music", "Music_For_Game.mp3")
    if os.path.exists(music_file):
        pygame.mixer.music.load(music_file)
        pygame.mixer.music.play(-1)
        pygame.mixer.music.set_volume(0)

    # Load enemy images once
    Enemy.load_images(config.enemy_image_vertical, config.enemy_image_horizontal, config.enemy_image_size)

    # Load player image
    load_player_image()

    # Player starting position
    start_x = config.WIDTH - config.wall_thickness - config.PLAYER_RADIUS
    start_y = config.HEIGHT - config.wall_thickness - config.PLAYER_RADIUS
    player_x, player_y = start_x, start_y

    # Level system
    current_level = 0
    computer_rect_pos, enemies, walls, player_start = load_level(screen, current_level)
    player_x, player_y = player_start
    computer_cooldown = False 
    running = True

    while running:
        dt = clock.tick(config.FPS) / 1000.0

        keys = pygame.key.get_pressed()
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                running = False
                show_game_over(screen)

            if event.key == pygame.K_l:
                current_level += 1
                if current_level >= len(LEVELS):
                    current_level = len(LEVELS) - 1
                computer_rect_pos, enemies, walls, player_x, player_y, computer_cooldown, player_start = load_level_safe(current_level, screen)


            if event.key == pygame.K_k:
                current_level -= 1
                if current_level < 0:
                    current_level = 0
                computer_rect_pos, enemies, walls, player_x, player_y, computer_cooldown, player_start = load_level_safe(current_level, screen)




        # Move player
        dx = dy = 0
        if keys[pygame.K_LEFT] or keys[pygame.K_a]: dx = -1
        if keys[pygame.K_RIGHT] or keys[pygame.K_d]: dx = 1
        if keys[pygame.K_UP] or keys[pygame.K_w]: dy = -1
        if keys[pygame.K_DOWN] or keys[pygame.K_s]: dy = 1
        if dx and dy: dx *= 0.7071; dy *= 0.7071

        new_x = player_x + dx * config.PLAYER_SPEED * dt
        new_y = player_y + dy * config.PLAYER_SPEED * dt
        future_rect = pygame.Rect(new_x - config.PLAYER_RADIUS, new_y - config.PLAYER_RADIUS, config.PLAYER_RADIUS*2, config.PLAYER_RADIUS*2)

        if not any(future_rect.colliderect(w) for w in walls):
            player_x, player_y = new_x, new_y

        # Update enemies
        for enemy in enemies:
            enemy.update(dt, walls)

        # Draw everything
        screen.fill(config.BG_COLOR)
        for enemy in enemies: enemy.draw(screen)
        player_rect = draw_player(screen, player_x, player_y)
        for w in walls:
            pygame.draw.rect(screen, config.wall_color, w)

        # Draw computer
        pygame.draw.rect(screen, (194, 194, 194), computer_rect_pos)

        # Collision with enemies
        for enemy in enemies:
            hitbox_padding = 15  # increase for closer detection
            enemy_rect = pygame.Rect(
            enemy.x + hitbox_padding,
            enemy.y + hitbox_padding,
            enemy.size[0] - hitbox_padding*2,
            enemy.size[1] - hitbox_padding*2
)
            if player_rect.colliderect(enemy_rect):
                player_x, player_y = player_start 
                break

        # Collision with computer
        if player_rect.colliderect(computer_rect_pos):
            if not computer_cooldown:
                computer_cooldown = True

                # show objectives left
                remaining = len(LEVELS) - current_level
                count_objectives(screen, remaining)

                wait_for_escape()
                pygame.event.clear()

                # Go to next level
                current_level += 1
                if current_level >= len(LEVELS):
                    show_escape_message(screen)
                    return
                else:
                    computer_rect_pos, enemies, walls, player_x, player_y, computer_cooldown, player_start = load_level_safe(current_level, screen)


        if not player_rect.colliderect(computer_rect_pos):
            computer_cooldown = False

        pygame.display.flip()

    pygame.quit()

if __name__ == "__main__":
    main()

: 